In [1]:
import pandas as pd

# load train data
df = pd.read_csv('train.csv')
df.shape

(76020, 371)

In [2]:
# since we have a reasonably training set, it makes sense to split it into a new training and test set
from sklearn.cross_validation import train_test_split

train, test = train_test_split(df, train_size=66020, test_size=10000)

In [3]:
# preparing data for training
Y_values = train.TARGET
train.drop('TARGET', axis=1, inplace=True)

# because we split test from the train
Original_Y_for_test = test.TARGET
test.drop('TARGET', axis=1, inplace=True)

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

ada_boost = AdaBoostClassifier(base_estimator=RandomForestClassifier())
ada_boost.fit(train, Y_values)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=None)

In [17]:
# predictions
ada_boost_predictions = ada_boost.predict(test)

In [18]:
# confusion matrix
from sklearn import metrics
metrics.confusion_matrix(Original_Y_for_test, ada_boost_predictions)

array([[9520,   68],
       [ 396,   16]])

In [20]:
# run random forest on the actual test set.
test_data_set = pd.read_csv('test.csv')
predictions_for_actual_test_data = ada_boost.predict(test_data_set)

In [22]:
# export the predictions to csv, to upload to kaggle
import csv

id_prediction_mappings = zip(test_data_set.ID, predictions_for_actual_test_data)
columns = ['ID','TARGET']

with open('aba_boost_results.csv', 'w') as csv_file:
    csv_writer = csv.DictWriter(csv_file, delimiter=',', fieldnames=columns)
    csv_writer.writeheader()
    
    for mapping in id_prediction_mappings:
        csv_writer.writerow({'ID': mapping[0], 'TARGET': mapping[1]})

In [ ]:
# kaggle score 0.513864